# Benin Solar Data - Exploratory Data Analysis

**Dataset:** benin-malanville.csv  
**Objective:** Profile, clean, and explore the Benin solar dataset to identify key trends and insights for solar investment decisions.

## Table of Contents
1. Setup & Data Loading
2. Summary Statistics & Missing Values
3. Outlier Detection & Data Quality
4. Data Cleaning
5. Time Series Analysis
6. Cleaning Impact Analysis
7. Correlation & Relationship Analysis
8. Wind Analysis & Distributions
9. Temperature & Humidity Analysis
10. Bubble Chart Analysis
11. Key Insights & Conclusions


## 1. Setup & Data Loading


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Figure size defaults
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")


Libraries imported successfully!


In [ ]:
# Load the dataset
data_path = '../data/benin-malanville.csv'
df = pd.read_csv(data_path)

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")


In [ ]:
# Initial data inspection
print("=" * 80)
print("FIRST 5 ROWS:")
print("=" * 80)
display(df.head())

print("\n" + "=" * 80)
print("LAST 5 ROWS:")
print("=" * 80)
display(df.tail())

print("\n" + "=" * 80)
print("DATA TYPES:")
print("=" * 80)
display(df.dtypes)


## 2. Summary Statistics & Missing Values


In [ ]:
# Summary statistics for all numeric columns
print("=" * 80)
print("SUMMARY STATISTICS FOR NUMERIC COLUMNS")
print("=" * 80)
summary_stats = df.describe()
display(summary_stats)


In [ ]:
# Missing values analysis
print("=" * 80)
print("MISSING VALUES ANALYSIS")
print("=" * 80)

missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Count': missing_values.values,
    'Missing Percentage': missing_percentage.values
})

missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Percentage', ascending=False)

if len(missing_df) > 0:
    display(missing_df)
    
    # Flag columns with >5% missing values
    high_missing = missing_df[missing_df['Missing Percentage'] > 5]
    if len(high_missing) > 0:
        print("\n⚠️ Columns with >5% missing values:")
        for col in high_missing['Column']:
            print(f"  - {col}: {high_missing[high_missing['Column']==col]['Missing Percentage'].values[0]:.2f}%")
else:
    print("✓ No missing values found in the dataset!")


## 3. Outlier Detection & Data Quality


In [ ]:
# Check for negative values in solar radiation columns
print("=" * 80)
print("DATA QUALITY CHECK: NEGATIVE VALUES IN SOLAR RADIATION")
print("=" * 80)

solar_cols = ['GHI', 'DNI', 'DHI']
for col in solar_cols:
    if col in df.columns:
        negative_count = (df[col] < 0).sum()
        negative_pct = (negative_count / len(df)) * 100
        print(f"{col}: {negative_count} negative values ({negative_pct:.2f}%)")
        
print("\nNote: Negative values during nighttime are expected, but should be investigated for daytime readings.")


In [ ]:
# Compute Z-scores for outlier detection
print("=" * 80)
print("OUTLIER DETECTION USING Z-SCORES (|Z| > 3)")
print("=" * 80)

# Columns to check for outliers
outlier_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Create a copy for outlier analysis
df_outlier = df.copy()

# Store outlier information
outlier_summary = []

for col in outlier_cols:
    if col in df.columns:
        # Calculate Z-scores
        z_scores = np.abs(stats.zscore(df[col].dropna()))
        
        # Count outliers (|Z| > 3)
        outliers = np.sum(z_scores > 3)
        outlier_pct = (outliers / len(df[col].dropna())) * 100
        
        outlier_summary.append({
            'Column': col,
            'Outliers': outliers,
            'Outlier Percentage': outlier_pct
        })

outlier_df = pd.DataFrame(outlier_summary)
display(outlier_df)


## 4. Data Cleaning


In [ ]:
# Create a cleaned copy of the dataset
df_clean = df.copy()

print("=" * 80)
print("DATA CLEANING PROCESS")
print("=" * 80)

print(f"\nOriginal dataset shape: {df_clean.shape}")

# Step 1: Handle missing values in key columns with median imputation
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'RH', 'WS', 'WSgust', 'WD', 'BP']

for col in key_columns:
    if col in df_clean.columns:
        missing_before = df_clean[col].isnull().sum()
        if missing_before > 0:
            median_val = df_clean[col].median()
            df_clean[col].fillna(median_val, inplace=True)
            print(f"✓ {col}: Imputed {missing_before} missing values with median ({median_val:.2f})")

# Step 2: Remove rows with critical missing data (Comments can be missing)
# Only drop rows where all key sensor readings are missing
critical_cols = ['GHI', 'DNI', 'DHI']
rows_before = len(df_clean)
df_clean = df_clean.dropna(subset=critical_cols, how='all')
rows_after = len(df_clean)
rows_dropped = rows_before - rows_after

if rows_dropped > 0:
    print(f"\n✓ Dropped {rows_dropped} rows with all critical solar radiation values missing")

print(f"\nCleaned dataset shape: {df_clean.shape}")
print(f"Rows removed: {len(df) - len(df_clean)}")
print(f"Remaining missing values: {df_clean.isnull().sum().sum()}")


In [ ]:
# Export cleaned dataset
output_path = '../data/benin-malanville_clean.csv'
df_clean.to_csv(output_path, index=False)
print(f"✓ Cleaned dataset exported to: {output_path}")
print(f"✓ File size: {len(df_clean)} rows × {len(df_clean.columns)} columns")


## 5. Time Series Analysis


In [ ]:
# Convert Timestamp to datetime
df_clean['Timestamp'] = pd.to_datetime(df_clean['Timestamp'])
df_clean.set_index('Timestamp', inplace=True)

print("✓ Timestamp converted to datetime index")
print(f"Date range: {df_clean.index.min()} to {df_clean.index.max()}")


In [ ]:
# Time series plot: Solar Radiation (GHI, DNI, DHI) over time
# Sample data for visualization (every 100th point to avoid overcrowding)
sample_df = df_clean.iloc[::100]

fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# GHI
axes[0].plot(sample_df.index, sample_df['GHI'], color='orange', linewidth=0.8)
axes[0].set_title('Global Horizontal Irradiance (GHI) Over Time', fontsize=12, fontweight='bold')
axes[0].set_ylabel('GHI (W/m²)', fontsize=10)
axes[0].grid(True, alpha=0.3)

# DNI
axes[1].plot(sample_df.index, sample_df['DNI'], color='red', linewidth=0.8)
axes[1].set_title('Direct Normal Irradiance (DNI) Over Time', fontsize=12, fontweight='bold')
axes[1].set_ylabel('DNI (W/m²)', fontsize=10)
axes[1].grid(True, alpha=0.3)

# DHI
axes[2].plot(sample_df.index, sample_df['DHI'], color='blue', linewidth=0.8)
axes[2].set_title('Diffuse Horizontal Irradiance (DHI) Over Time', fontsize=12, fontweight='bold')
axes[2].set_ylabel('DHI (W/m²)', fontsize=10)
axes[2].set_xlabel('Date', fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Time series plot: Temperature over time
plt.figure(figsize=(14, 5))
plt.plot(sample_df.index, sample_df['Tamb'], color='green', linewidth=0.8)
plt.title('Ambient Temperature (Tamb) Over Time', fontsize=14, fontweight='bold')
plt.ylabel('Temperature (°C)', fontsize=12)
plt.xlabel('Date', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Monthly trend analysis
df_clean['Month'] = df_clean.index.month
df_clean['Hour'] = df_clean.index.hour

# Monthly averages
monthly_avg = df_clean.groupby('Month')[['GHI', 'DNI', 'DHI', 'Tamb']].mean()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Solar radiation by month
monthly_avg[['GHI', 'DNI', 'DHI']].plot(kind='bar', ax=axes[0], color=['orange', 'red', 'blue'])
axes[0].set_title('Average Solar Radiation by Month', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Month', fontsize=10)
axes[0].set_ylabel('Irradiance (W/m²)', fontsize=10)
axes[0].legend(['GHI', 'DNI', 'DHI'])
axes[0].grid(True, alpha=0.3)

# Temperature by month
monthly_avg['Tamb'].plot(kind='bar', ax=axes[1], color='green')
axes[1].set_title('Average Temperature by Month', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Month', fontsize=10)
axes[1].set_ylabel('Temperature (°C)', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Hourly (daily) pattern analysis
hourly_avg = df_clean.groupby('Hour')[['GHI', 'DNI', 'DHI', 'Tamb']].mean()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Solar radiation by hour
axes[0].plot(hourly_avg.index, hourly_avg['GHI'], marker='o', label='GHI', color='orange')
axes[0].plot(hourly_avg.index, hourly_avg['DNI'], marker='s', label='DNI', color='red')
axes[0].plot(hourly_avg.index, hourly_avg['DHI'], marker='^', label='DHI', color='blue')
axes[0].set_title('Average Solar Radiation by Hour of Day', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Hour of Day', fontsize=10)
axes[0].set_ylabel('Irradiance (W/m²)', fontsize=10)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Temperature by hour
axes[1].plot(hourly_avg.index, hourly_avg['Tamb'], marker='o', color='green', linewidth=2)
axes[1].set_title('Average Temperature by Hour of Day', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Hour of Day', fontsize=10)
axes[1].set_ylabel('Temperature (°C)', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Key Observations:")
print(f"  - Peak GHI occurs around hour {hourly_avg['GHI'].idxmax()}")
print(f"  - Peak temperature occurs around hour {hourly_avg['Tamb'].idxmax()}")


## 6. Cleaning Impact Analysis


In [ ]:
# Analyze impact of cleaning on sensor readings (ModA and ModB)
print("=" * 80)
print("CLEANING IMPACT ON SENSOR READINGS")
print("=" * 80)

# Group by cleaning flag
cleaning_impact = df_clean.groupby('Cleaning')[['ModA', 'ModB']].mean()

print("\nAverage Sensor Readings by Cleaning Flag:")
display(cleaning_impact)

# Visualize the impact
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ModA comparison
cleaning_impact['ModA'].plot(kind='bar', ax=axes[0], color=['coral', 'skyblue'])
axes[0].set_title('Average ModA: Before vs After Cleaning', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Cleaning Flag (0=Before, 1=After)', fontsize=10)
axes[0].set_ylabel('ModA Average', fontsize=10)
axes[0].set_xticklabels(['Not Cleaned (0)', 'Cleaned (1)'], rotation=0)
axes[0].grid(True, alpha=0.3)

# ModB comparison
cleaning_impact['ModB'].plot(kind='bar', ax=axes[1], color=['coral', 'skyblue'])
axes[1].set_title('Average ModB: Before vs After Cleaning', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Cleaning Flag (0=Before, 1=After)', fontsize=10)
axes[1].set_ylabel('ModB Average', fontsize=10)
axes[1].set_xticklabels(['Not Cleaned (0)', 'Cleaned (1)'], rotation=0)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate percentage difference
if 0 in cleaning_impact.index and 1 in cleaning_impact.index:
    pct_diff_moda = ((cleaning_impact.loc[1, 'ModA'] - cleaning_impact.loc[0, 'ModA']) / cleaning_impact.loc[0, 'ModA']) * 100
    pct_diff_modb = ((cleaning_impact.loc[1, 'ModB'] - cleaning_impact.loc[0, 'ModB']) / cleaning_impact.loc[0, 'ModB']) * 100
    
    print(f"\n📊 Impact of Cleaning:")
    print(f"  - ModA change: {pct_diff_moda:.2f}%")
    print(f"  - ModB change: {pct_diff_modb:.2f}%")


## 7. Correlation & Relationship Analysis


In [ ]:
# Correlation heatmap for key variables
correlation_vars = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB']
corr_matrix = df_clean[correlation_vars].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            fmt='.2f', vmin=-1, vmax=1)
plt.title('Correlation Heatmap: Solar Radiation & Module Temperature', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Key correlations identified in the heatmap above.")


In [ ]:
# Scatter plots: Wind conditions vs Solar Radiation
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Sample data for scatter plots (use every 10th point to avoid overcrowding)
scatter_sample = df_clean.iloc[::10]

# WS vs GHI
axes[0, 0].scatter(scatter_sample['WS'], scatter_sample['GHI'], alpha=0.5, s=10, color='blue')
axes[0, 0].set_title('Wind Speed (WS) vs GHI', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Wind Speed (m/s)', fontsize=10)
axes[0, 0].set_ylabel('GHI (W/m²)', fontsize=10)
axes[0, 0].grid(True, alpha=0.3)
# Add correlation coefficient
corr_ws_ghi = df_clean['WS'].corr(df_clean['GHI'])
axes[0, 0].text(0.05, 0.95, f'Corr: {corr_ws_ghi:.3f}', transform=axes[0, 0].transAxes, 
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# WSgust vs GHI
axes[0, 1].scatter(scatter_sample['WSgust'], scatter_sample['GHI'], alpha=0.5, s=10, color='orange')
axes[0, 1].set_title('Wind Gust (WSgust) vs GHI', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Wind Gust (m/s)', fontsize=10)
axes[0, 1].set_ylabel('GHI (W/m²)', fontsize=10)
axes[0, 1].grid(True, alpha=0.3)
corr_wsg_ghi = df_clean['WSgust'].corr(df_clean['GHI'])
axes[0, 1].text(0.05, 0.95, f'Corr: {corr_wsg_ghi:.3f}', transform=axes[0, 1].transAxes, 
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# WD vs GHI
axes[1, 0].scatter(scatter_sample['WD'], scatter_sample['GHI'], alpha=0.5, s=10, color='green')
axes[1, 0].set_title('Wind Direction (WD) vs GHI', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Wind Direction (°)', fontsize=10)
axes[1, 0].set_ylabel('GHI (W/m²)', fontsize=10)
axes[1, 0].grid(True, alpha=0.3)
corr_wd_ghi = df_clean['WD'].corr(df_clean['GHI'])
axes[1, 0].text(0.05, 0.95, f'Corr: {corr_wd_ghi:.3f}', transform=axes[1, 0].transAxes, 
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# RH vs Tamb
axes[1, 1].scatter(scatter_sample['RH'], scatter_sample['Tamb'], alpha=0.5, s=10, color='red')
axes[1, 1].set_title('Relative Humidity (RH) vs Temperature', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Relative Humidity (%)', fontsize=10)
axes[1, 1].set_ylabel('Temperature (°C)', fontsize=10)
axes[1, 1].grid(True, alpha=0.3)
corr_rh_tamb = df_clean['RH'].corr(df_clean['Tamb'])
axes[1, 1].text(0.05, 0.95, f'Corr: {corr_rh_tamb:.3f}', transform=axes[1, 1].transAxes, 
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()


In [ ]:
# Additional scatter plot: RH vs GHI
plt.figure(figsize=(10, 6))
plt.scatter(scatter_sample['RH'], scatter_sample['GHI'], alpha=0.5, s=15, color='purple')
plt.title('Relative Humidity (RH) vs Global Horizontal Irradiance (GHI)', fontsize=14, fontweight='bold')
plt.xlabel('Relative Humidity (%)', fontsize=12)
plt.ylabel('GHI (W/m²)', fontsize=12)
plt.grid(True, alpha=0.3)

# Add correlation
corr_rh_ghi = df_clean['RH'].corr(df_clean['GHI'])
plt.text(0.05, 0.95, f'Correlation: {corr_rh_ghi:.3f}', transform=plt.gca().transAxes, 
         verticalalignment='top', fontsize=12, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()


## 8. Wind Analysis & Distributions


In [ ]:
# Wind Rose: Polar plot showing wind speed by direction
# Bin wind directions into 16 sectors (N, NNE, NE, ENE, E, etc.)
wind_data = df_clean[['WD', 'WS']].copy()
wind_data = wind_data[wind_data['WS'] > 0]  # Filter out zero wind speed

# Create direction bins
direction_bins = np.arange(0, 360, 22.5)  # 16 sectors
wind_data['WD_bin'] = pd.cut(wind_data['WD'], bins=np.append(direction_bins, 360), 
                               labels=direction_bins, include_lowest=True)

# Calculate average wind speed per direction
wind_by_direction = wind_data.groupby('WD_bin')['WS'].mean()

# Create polar plot
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='polar')

# Convert degrees to radians
theta = np.deg2rad(wind_by_direction.index.astype(float))
radii = wind_by_direction.values

# Create bars
bars = ax.bar(theta, radii, width=np.deg2rad(22.5), bottom=0, alpha=0.7, color='skyblue', edgecolor='navy')

# Customize
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)
ax.set_title('Wind Rose: Average Wind Speed by Direction', fontsize=14, fontweight='bold', pad=20)
ax.set_ylabel('Wind Speed (m/s)', fontsize=11)

plt.tight_layout()
plt.show()

print(f"Predominant wind direction: {wind_by_direction.idxmax()}°")
print(f"Average wind speed from that direction: {wind_by_direction.max():.2f} m/s")


In [ ]:
# Histograms for key variables
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# GHI distribution
axes[0, 0].hist(df_clean['GHI'], bins=50, color='orange', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Distribution of Global Horizontal Irradiance (GHI)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('GHI (W/m²)', fontsize=10)
axes[0, 0].set_ylabel('Frequency', fontsize=10)
axes[0, 0].axvline(df_clean['GHI'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df_clean["GHI"].mean():.2f}')
axes[0, 0].axvline(df_clean['GHI'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df_clean["GHI"].median():.2f}')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# WS distribution
axes[0, 1].hist(df_clean['WS'], bins=50, color='blue', alpha=0.7, edgecolor='black')
axes[0, 1].set_title('Distribution of Wind Speed (WS)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Wind Speed (m/s)', fontsize=10)
axes[0, 1].set_ylabel('Frequency', fontsize=10)
axes[0, 1].axvline(df_clean['WS'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df_clean["WS"].mean():.2f}')
axes[0, 1].axvline(df_clean['WS'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df_clean["WS"].median():.2f}')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Temperature distribution
axes[1, 0].hist(df_clean['Tamb'], bins=50, color='green', alpha=0.7, edgecolor='black')
axes[1, 0].set_title('Distribution of Ambient Temperature', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Temperature (°C)', fontsize=10)
axes[1, 0].set_ylabel('Frequency', fontsize=10)
axes[1, 0].axvline(df_clean['Tamb'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df_clean["Tamb"].mean():.2f}')
axes[1, 0].axvline(df_clean['Tamb'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df_clean["Tamb"].median():.2f}')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# RH distribution
axes[1, 1].hist(df_clean['RH'], bins=50, color='purple', alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Distribution of Relative Humidity (RH)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Relative Humidity (%)', fontsize=10)
axes[1, 1].set_ylabel('Frequency', fontsize=10)
axes[1, 1].axvline(df_clean['RH'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df_clean["RH"].mean():.2f}')
axes[1, 1].axvline(df_clean['RH'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df_clean["RH"].median():.2f}')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
